<a href="https://colab.research.google.com/github/Shivang696/university-query-priority-classification/blob/main/Classification_university_query_priority.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
train_data='https://raw.githubusercontent.com/coderanandmaurya/University-Query-Priority-Classification/refs/heads/main/university_query_train.csv'
test_data='https://raw.githubusercontent.com/coderanandmaurya/University-Query-Priority-Classification/refs/heads/main/university_query_test.csv'

In [ ]:
import pandas as pd
train=pd.read_csv(train_data)
test=pd.read_csv(test_data)

In [ ]:
train_df=pd.read_csv(train_data,usecols=['Student_Query', 'Department', 'Days_To_Deadline',
       'Priority_Label'])
test_df=pd.read_csv(test_data,usecols=['Student_Query', 'Department', 'Days_To_Deadline',
       'Priority_Label'])

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          5000 non-null   int64 
 1   Student_Query     5000 non-null   object
 2   Department        5000 non-null   object
 3   Days_To_Deadline  5000 non-null   int64 
 4   Priority_Label    5000 non-null   object
dtypes: int64(2), object(3)
memory usage: 195.4+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          1000 non-null   int64 
 1   Student_Query     1000 non-null   object
 2   Department        1000 non-null   object
 3   Days_To_Deadline  1000 non-null   int64 
 4   Priority_Label    1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [ ]:
train.sample()

,Query_ID,Student_Query,Department,Days_To_Deadline,Priority_Label
2410,2896,Request for internship approval letter.,Examination Cell,36,Medium


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

In [ ]:
# find max len of text in data
max_len = train_df["Student_Query"].apply(lambda x: len(x.split())).max()
max_len

12

In [ ]:
max_words = 10000
max_len = 15

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["Student_Query"])

X_train_text = tokenizer.texts_to_sequences(train_df["Student_Query"])
X_test_text = tokenizer.texts_to_sequences(test_df["Student_Query"])

X_train_text = pad_sequences(X_train_text, maxlen=max_len, padding="post")
X_test_text = pad_sequences(X_test_text, maxlen=max_len, padding="post")

In [ ]:
X_train_text

array([[13,  8, 55, ...,  0,  0,  0],
       [ 3, 74, 75, ...,  0,  0,  0],
       [13,  8, 30, ...,  0,  0,  0],
       ...,
       [12, 32, 11, ...,  0,  0,  0],
       [13,  8, 30, ...,  0,  0,  0],
       [19, 20, 21, ...,  0,  0,  0]], dtype=int32)

In [ ]:
dept_encoder = LabelEncoder()

train_df["Department"] = dept_encoder.fit_transform(train_df["Department"])
test_df["Department"] = dept_encoder.transform(test_df["Department"])

num_departments = len(dept_encoder.classes_)

In [ ]:
scaler = StandardScaler()

X_train_days = scaler.fit_transform(train_df[["Days_To_Deadline"]])
X_test_days = scaler.transform(test_df[["Days_To_Deadline"]])

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(train_df["Priority_Label"])
y_test = label_encoder.transform(test_df["Priority_Label"])

num_classes = len(label_encoder.classes_)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
text_input = Input(shape=(max_len,), name="text_input")

embedding = Embedding(input_dim=max_words,
                      output_dim=128,
                      input_length=max_len)(text_input)

lstm_out = LSTM(64)(embedding)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
dept_input = Input(shape=(1,), name="dept_input")

dept_embedding = Embedding(input_dim=num_departments,
                           output_dim=8)(dept_input)

dept_flat = tf.keras.layers.Flatten()(dept_embedding)

In [ ]:
days_input = Input(shape=(1,), name="days_input")

In [ ]:
concat = Concatenate()([lstm_out, dept_flat, days_input])

dense = Dense(64, activation="relu")(concat)
drop = Dropout(0.3)(dense)

output = Dense(num_classes, activation="softmax")(drop)

In [ ]:
model = Model(
    inputs=[text_input, dept_input, days_input],
    outputs=output
)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 15, 128)   │  1,280,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 8)      │         56 │ dept_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ days_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 73)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ days_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,736 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        195 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,334,395 (5.09 MB)

 Trainable params: 1,334,395 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    [X_train_text, train_df["Department"], X_train_days],
    y_train,
    validation_data=(
        [X_test_text, test_df["Department"], X_test_days],
        y_test
    ),
    epochs=10,
    batch_size=32
)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.7315 - loss: 0.5221 - val_accuracy: 1.0000 - val_loss: 5.4582e-05
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 1.0000 - loss: 3.4190e-04 - val_accuracy: 1.0000 - val_loss: 1.3890e-05
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 1.0000 - loss: 1.1594e-04 - val_accuracy: 1.0000 - val_loss: 5.5049e-06
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 8.9370e-05 - val_accuracy: 1.0000 - val_loss: 2.8703e-06
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 4.1921e-05 - val_accuracy: 1.0000 - val_loss: 1.7031e-06
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 1.0000 - loss: 3.2581e-05 - val_accuracy: 1.0000 - val_loss: 1.0434e-06
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 3.6404e-05 - val_accuracy: 1.0000 - val_loss: 6.9356e-07
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20